In [0]:
def commentSentimentAnalysis(pysparkDf, colName):
  """
  This functions performs the sentiment analysis on the passed pyspark dataframe. We use cardiffnlp/twitter-xlm-roberta-base-sentiment model from huggingface for the scoring. Passed pyspark dataframe should have a column called `comment` and if the schema of the passed dataframe dont match, the functions returns the result else the result is stored in the predetermined location as a delta table. The expected dataframe should have the these columns title:string, description:string, tags:string, videoId:string, commentId:string, commentUpdatedDate:date, commentAuthorChannelId:string, comment:string

  usage:
  
  commentSentimentAnalysis(pysparkDf = youtubeVideoCommentsRepliesFiltered)
  
  """
  from pyspark.sql import SparkSession
  from pyspark.sql.functions import col, udf
  from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField
  from pyspark.storagelevel import StorageLevel
  from pyspark.sql import SparkSession
  spark = SparkSession.builder.getOrCreate()
  
  #destination path
  sentimentsMountPath = "/mnt/Sandbox/pm_ap_conn/sid_test/youtube/commentSentimentsTest202401060630"
  #tweetnlp function load
  def getSentimentModel():
    import tweetnlp
    model = tweetnlp.load_model('sentiment', multilingual=True)
    return model
  
  def sentimentUdf(model, text):
    # Perform sentiment analysis on the entire text
    x = model.sentiment(text, return_probability=True, batch_size=32)
    label = x['label']
    probability = x['probability'][label]
    sentiment_details = {"label": label, "probability": probability}
    return sentiment_details

  def main(pysparkDf, colName):
    #load the model only once
    sentimentModel = getSentimentModel()
    #read input dataset
    feed = pysparkDf.repartition(31)
    #define UDF for sentiment analysis
    sentimentSchema = MapType(StringType(), StringType())
    sentimentSparkUdf = udf(lambda comments: sentimentUdf(sentimentModel, comments), sentimentSchema)

    #apply the UDF to the spark dataFrame
    result = feed.withColumn("sentimentAnalysis", sentimentSparkUdf(col(colName))) \
                        .withColumn("sentimentCategory", col("sentimentAnalysis").getItem("label")) \
                        .withColumn("sentimentCategoryProbability", col("sentimentAnalysis").getItem("probability"))
    #save the result as delta format in datalake
    try:
      result.write.format("delta").mode("overwrite").option("overwriteSchema", "False").save(sentimentsMountPath)
    except:
      print("schema of the passed df dont meet the requirements. Hence returned!!")
    return result

  main(pysparkDf, colName) 

In [0]:
pip install -U tensorflow==2.10 

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached tensorflow-2.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.0 MB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Not uninstalling keras at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/

In [0]:
pip install tweetnlp==0.4.4

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 143.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 143.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 73.4 MB/s

In [0]:
display(commentSentimentAnalysis('2024-08-01', '2024-08-10', slang_dict_path = '/Workspace/Users/siddardha.kaja@aircanada.ca/EnglishSDK/slangDict.txt'))

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-1330541243429584>, line 1
----> 1 display(commentSentimentAnalysis('2024-08-01', '2024-08-10', slang_dict_path = '/Workspace/Users/siddardha.kaja@aircanada.ca/EnglishSDK/slangDict.txt'))

File <command-1330541243429580>, line 135, in commentSentimentAnalysis(startCommentDate, endCommentDate, slang_dict_path)
    133   finalDf = commentSentimentAnalysis(forSentiAnalysis, "expandedText")
    134   return finalDf
--> 135 main(startCommentDate, endCommentDate, slang_dict_path)

File <command-1330541243429580>, line 133, in commentSentimentAnalysis.<locals>.main(startCommentDate, endCommentDate, slang_dict_path)
    130 forSentiAnalysis = cleanedTextDf.union(langFilteredNonEN_1)
    132 #do sentiment analysis
--> 133 finalDf = commentSentimentAnalysis(forSentiAnalysis, "expandedText")
    134 return finalDf

TypeError: commentS